# Custom Workflows and State Standards with RasCheck

This notebook demonstrates advanced RasCheck workflows:

- **State-specific floodway standards** (all 50 states)
- **Custom threshold configuration**
- **Batch processing patterns**
- **CSV export workflows**
- **Integration with external tools**

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = True  # <-- Set to True for check module (in development)

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")
    print("⚠️  WARNING: Check module may not be available in pip package yet")

# Import ras-commander
from ras_commander import HdfResultsPlan, RasCheck, RasCmdr, RasExamples, init_ras_project, ras

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

## Parameters

Configure these values to customize the notebook for your project.

In [ ]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "Muncie"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Output Configuration
OUTPUTS_DIR = Path("_outputs") / "802_custom_workflows_and_standards"  # Artifacts saved here

# Execution Settings
PLAN = "01"                       # Plan number to execute
NUM_CORES = 4                     # CPU cores for 2D computation
RUN_SUFFIX = "run"                # Suffix for run folder (e.g., Muncie_run)

# Create output directory
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Outputs will be saved to: {OUTPUTS_DIR.absolute()}")


### State-Specific QA Standards

**When implementing state-specific workflows**:

- Document which state standards apply (IL, TX, WI, etc.)
- Reference specific code sections or guidance documents
- Justify any deviations from default FEMA standards
- Include in engineering analysis documentation

**Common State Variations**:
- **Illinois**: 0.1 ft floodway surcharge limit (stricter than default 1.0 ft)
- **Texas**: TNRIS floodplain mapping requirements
- **Wisconsin**: DNR floodplain standards (NR 116)
- **Minnesota**: DNR floodplain rule (6115)

**References**:
- State floodplain management regulations (varies by state)
- Local drainage district standards
- Municipal floodplain ordinances

## Import Required Modules

In [ ]:
from ras_commander import (
    RasExamples, init_ras_project, ras,
    RasCmdr, HdfResultsPlan
)
from ras_commander.check import (
    RasCheck, CheckResults, Severity,
    RasCheckReport, ReportMetadata,
    get_default_thresholds, create_custom_thresholds,
    get_state_surcharge_limit, ValidationThresholds
)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Set pandas display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

---

## Part 1: State-Specific Floodway Standards

Different states have different maximum surcharge limits for floodway analysis. RasCheck supports all 50 states plus territories.

In [ ]:
# Get all US state codes
us_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Create comprehensive state standards table
state_standards = []
for state in us_states:
    limit = get_state_surcharge_limit(state)
    state_standards.append({
        'State': state,
        'Max Surcharge (ft)': limit,
        'Standard Type': 'Stricter' if limit < 1.0 else 'Standard'
    })

df_standards = pd.DataFrame(state_standards)

print("US State Floodway Surcharge Limits:")
print("="*60)
display(df_standards)

In [ ]:
# Visualize state standards
fig, ax = plt.subplots(figsize=(14, 8))

# Color code by standard type
colors = ['red' if limit < 1.0 else 'steelblue' for limit in df_standards['Max Surcharge (ft)']]

ax.barh(df_standards['State'], df_standards['Max Surcharge (ft)'], color=colors, alpha=0.7)
ax.axvline(1.0, color='black', linestyle='--', linewidth=2, label='FEMA Standard (1.0 ft)')
ax.set_xlabel('Maximum Surcharge (ft)', fontsize=12)
ax.set_ylabel('State', fontsize=12)
ax.set_title('State-Specific Floodway Surcharge Limits', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')

# Add annotation for stricter states
stricter_states = df_standards[df_standards['Max Surcharge (ft)'] < 1.0]['State'].tolist()
ax.text(0.05, 0.95, f"Stricter Standards ({len(stricter_states)}): {', '.join(stricter_states)}",
        transform=ax.transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

### States with Stricter Standards

Some states require more stringent floodway analysis:

| State | Limit | Notes |
|-------|-------|-------|
| **Illinois (IL)** | 0.1 ft | Most stringent |
| **Wisconsin (WI)** | 0.5 ft | Below FEMA standard |
| **Minnesota (MN)** | 0.5 ft | Below FEMA standard |
| **New Jersey (NJ)** | 0.5 ft | Below FEMA standard |
| **Michigan (MI)** | 0.5 ft | Below FEMA standard |
| **Indiana (IN)** | 0.5 ft | Below FEMA standard |
| **Ohio (OH)** | 0.5 ft | Below FEMA standard |

**Default (Most States)**: 1.0 ft (FEMA/NFIP standard)

---

## Part 2: Custom Threshold Configuration

RasCheck allows complete customization of validation thresholds for project-specific requirements.

In [ ]:
# View default thresholds structuredefaults = get_default_thresholds()print("Default Threshold Categories:")print("="*60)print(f"1. Manning's n Ranges:")print(f"   - Channel: {defaults.mannings_n.channel_min} to {defaults.mannings_n.channel_max}")print(f"   - Overbank: {defaults.mannings_n.overbank_min} to {defaults.mannings_n.overbank_max}")print(f"\n2. Transition Coefficients:")print(f"   - Regular: {defaults.transitions.regular_contraction_max} (C) / {defaults.transitions.regular_expansion_max} (E)")print(f"   - Structure: {defaults.transitions.structure_contraction_max} (C) / {defaults.transitions.structure_expansion_max} (E)")print(f"\n3. Reach Lengths:")print(f"   - Max length: {defaults.reach_length.max_length_ft} ft")print(f"   - Max ratio: {defaults.reach_length.length_ratio_max}")print(f"\n4. Culvert Coefficients:")print(f"   - Entrance: {defaults.structures.culvert_entrance_coef_min} to {defaults.structures.culvert_entrance_coef_max}")print(f"   - Exit: {defaults.structures.culvert_exit_coef_typical} (typical)")print(f"\n5. Floodway:")print(f"   - Max surcharge: {defaults.floodway.surcharge_max_ft} ft")

### Example 1: Highly Vegetated Watershed

For watersheds with extensive vegetation, use higher Manning's n thresholds:

In [ ]:
# Create custom thresholds for highly vegetated watershedvegetated_thresholds = create_custom_thresholds({    # Allow higher n values for vegetated channels    'mannings_n.channel_max': 0.100,  # Default: 0.080    'mannings_n.overbank_max': 0.200,  # Default: 0.150        # More conservative transition losses    'transitions.regular_contraction_max': 0.15,  # Default: 0.10    'transitions.regular_expansion_max': 0.35,    # Default: 0.30})print("✓ Custom thresholds created for vegetated watershed")print(f"  Channel n max: {vegetated_thresholds.mannings_n.channel_max}")print(f"  Overbank n max: {vegetated_thresholds.mannings_n.overbank_max}")

### Example 2: Urban Development Project

For urban areas with lined channels and close cross sections:

In [ ]:
# Create custom thresholds for urban project
urban_thresholds = create_custom_thresholds({
    # Tighter n ranges for lined channels
    'mannings_n.channel_min': 0.012,  # Default: 0.015
    'mannings_n.channel_max': 0.035,  # Default: 0.080
    'mannings_n.overbank_min': 0.015, # Default: 0.020
    'mannings_n.overbank_max': 0.060, # Default: 0.150
    
    # Stricter cross section spacing for urban
    'reach_length.max_length_ft': 500.0,  # Default: 2000.0
    'reach_length.length_ratio_max': 1.3,  # Default: 1.5
})

print("✓ Custom thresholds created for urban project")
print(f"  Channel n range: {urban_thresholds.mannings_n.channel_min} - {urban_thresholds.mannings_n.channel_max}")
print(f"  Max XS spacing: {urban_thresholds.reach_length.max_length_ft} ft")

### Example 3: State-Specific Floodway (Illinois)

Apply Illinois's stringent 0.1 ft surcharge limit:

In [ ]:
# Create thresholds for Illinois floodway analysisil_limit = get_state_surcharge_limit('IL')illinois_thresholds = create_custom_thresholds({    'floodway.surcharge_max_ft': il_limit  # 0.1 ft})print(f"✓ Custom thresholds created for Illinois")print(f"  Max surcharge: {illinois_thresholds.floodway.surcharge_max_ft} ft")print(f"  ⚠️  This is 10x stricter than FEMA standard (1.0 ft)")

---

## Part 3: Batch Processing Workflows

Process multiple models or scenarios efficiently:

In [ ]:
# Extract and setup test project
project_path = RasExamples.extract_project(PROJECT_NAME, 
                                           output_path=OUTPUTS_DIR.parent / "example_projects")
init_ras_project(project_path, RAS_VERSION)

print(f"✓ Initialized project: {ras.project_name}")
print(f"  Plans available: {len(ras.plan_df)}")
display(ras.plan_df[['plan_number', 'Plan Title', 'flow_type']])

### Workflow 1: Compare Multiple Threshold Scenarios

In [ ]:
# Define threshold scenarios to test
scenarios = {
    'FEMA Standard': get_default_thresholds(),
    'Vegetated Watershed': vegetated_thresholds,
    'Urban Development': urban_thresholds,
    'Illinois Floodway': illinois_thresholds
}

# Run validation with each scenario
plan_number = "02"  # Use steady flow plan

scenario_results = {}
for scenario_name, thresholds in scenarios.items():
    print(f"Running {scenario_name}...")
    results = RasCheck.run_all(plan_number, thresholds=thresholds)
    scenario_results[scenario_name] = results
    print(f"  {len(results.messages)} messages ({results.get_error_count()} errors, {results.get_warning_count()} warnings)")

In [ ]:
# Compare scenario results
comparison_data = []
for scenario_name, results in scenario_results.items():
    comparison_data.append({
        'Scenario': scenario_name,
        'Total Messages': len(results.messages),
        'Errors': results.get_error_count(),
        'Warnings': results.get_warning_count(),
        'Info': len([m for m in results.messages if m.severity == Severity.INFO])
    })

df_comparison = pd.DataFrame(comparison_data)
print("\nScenario Comparison:")
print("="*60)
display(df_comparison)

In [ ]:
# Visualize scenario comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_comparison))
width = 0.25

ax.bar(x - width, df_comparison['Errors'], width, label='Errors', color='red', alpha=0.8)
ax.bar(x, df_comparison['Warnings'], width, label='Warnings', color='orange', alpha=0.8)
ax.bar(x + width, df_comparison['Info'], width, label='Info', color='blue', alpha=0.8)

ax.set_xlabel('Scenario', fontsize=12)
ax.set_ylabel('Message Count', fontsize=12)
ax.set_title('Validation Results by Threshold Scenario', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(df_comparison['Scenario'], rotation=15, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### Workflow 2: Batch Export to CSV

In [ ]:
# Export all scenario results to CSV files
output_dir = project_path / "ras_checker" / "batch_results"
output_dir.mkdir(parents=True, exist_ok=True)

for scenario_name, results in scenario_results.items():
    # Create safe filename
    safe_name = scenario_name.replace(' ', '_').lower()
    csv_path = output_dir / f"{safe_name}_validation.csv"
    
    # Export to CSV
    df = results.to_dataframe()
    if not df.empty:
        df.to_csv(csv_path, index=False)
        print(f"✓ Exported {len(df)} messages to: {csv_path.name}")
    else:
        print(f"  No messages for {scenario_name}")

### Workflow 3: Automated Report Generation

In [ ]:
# Generate HTML reports for all scenarios
for scenario_name, results in scenario_results.items():
    safe_name = scenario_name.replace(' ', '_').lower()
    report_path = output_dir / f"{safe_name}_report.html"
    
    metadata = ReportMetadata(
        project_name=f"{ras.project_name} - {scenario_name}",
        project_path=project_path,
        plan_name=f"Scenario: {scenario_name}"
    )
    
    results.to_html(report_path, metadata=metadata)
    print(f"✓ Generated report: {report_path.name}")

print(f"\n✓ All reports saved to: {output_dir}")

---

## Part 4: Advanced CSV Analysis

Analyze validation results using pandas for custom insights:

In [ ]:
# Use FEMA standard results for analysis
results = scenario_results['FEMA Standard']
df = results.to_dataframe()

if not df.empty:
    print(f"Loaded {len(df)} validation messages for analysis")
    print(f"\nAvailable columns: {list(df.columns)}")
else:
    print("No messages available for analysis")

### Analysis 1: Identify Top Issues

In [ ]:
if not df.empty:
    # Count issues by message ID
    top_issues = df['message_id'].value_counts().head(10)
    
    print("Top 10 Most Common Issues:")
    print("="*60)
    for msg_id, count in top_issues.items():
        severity = df[df['message_id'] == msg_id]['severity'].iloc[0]
        print(f"  [{severity}] {msg_id}: {count} occurrences")

### Analysis 2: Spatial Distribution of Issues

In [ ]:
if not df.empty:
    # Group by river/reach
    if 'river' in df.columns and 'reach' in df.columns:
        spatial_summary = df.groupby(['river', 'reach']).agg({
            'message_id': 'count',
            'severity': lambda x: f"{(x=='ERROR').sum()}E / {(x=='WARNING').sum()}W"
        }).rename(columns={'message_id': 'Total Messages', 'severity': 'Breakdown'})
        
        print("\nIssues by River/Reach:")
        print("="*60)
        display(spatial_summary)

### Analysis 3: Check Type Performance

In [ ]:
if not df.empty:
    # Analyze by check type
    check_summary = df.groupby('check_type')['severity'].value_counts().unstack(fill_value=0)
    check_summary['Total'] = check_summary.sum(axis=1)
    check_summary = check_summary.sort_values('Total', ascending=False)
    
    print("\nValidation Results by Check Type:")
    print("="*60)
    display(check_summary)

---

## Part 5: Integration Patterns

### Pattern 1: Pre-Submission QA Workflow

In [ ]:
def pre_submission_qa(plan_number, state_code='TX', output_folder=None):    """    Complete QA workflow for FEMA submission.        Parameters    ----------    plan_number : str        HEC-RAS plan number to validate    state_code : str        State code for appropriate surcharge limit    output_folder : Path, optional        Output folder for reports (default: project folder)        Returns    -------    dict        Summary of validation results    """    print(f"Running Pre-Submission QA for Plan {plan_number}")    print(f"State: {state_code}")    print("="*60)        # 1. Get state-specific thresholds    surcharge = get_state_surcharge_limit(state_code)    thresholds = create_custom_thresholds({        'floodway.surcharge_max_ft': surcharge    })    print(f"✓ Using {state_code} surcharge limit: {surcharge} ft")        # 2. Run all checks    results = RasCheck.run_all(plan_number, thresholds=thresholds)    print(f"✓ Validation complete: {len(results.messages)} messages")        # 3. Generate outputs    if output_folder is None:        output_folder = project_path / "qa_submission"    output_folder.mkdir(parents=True, exist_ok=True)        # HTML report    html_path = output_folder / "validation_report.html"    metadata = ReportMetadata(        project_name=ras.project_name,        project_path=project_path,        plan_name=f"Plan {plan_number} - {state_code} Standards"    )    results.to_html(html_path, metadata=metadata)    print(f"✓ HTML report: {html_path}")        # CSV export    csv_path = output_folder / "validation_messages.csv"    df = results.to_dataframe()    if not df.empty:        df.to_csv(csv_path, index=False)        print(f"✓ CSV export: {csv_path}")        # 4. Return summary    summary = {        'plan': plan_number,        'state': state_code,        'total_messages': len(results.messages),        'errors': results.get_error_count(),        'warnings': results.get_warning_count(),        'ready_for_submission': results.get_error_count() == 0    }        print("\n" + "="*60)    print(f"QA Summary:")    print(f"  Errors: {summary['errors']}")    print(f"  Warnings: {summary['warnings']}")    print(f"  Ready for submission: {summary['ready_for_submission']}")        return summary# Test the workflowsummary = pre_submission_qa("02", state_code="TX")

### Pattern 2: Continuous Monitoring During Model Development

In [ ]:
def track_validation_progress(plan_number, iterations):
    """
    Track how validation improves over multiple model iterations.
    
    In practice, you would run this after each model revision.
    """
    progress = []
    
    for i in range(iterations):
        # In real use, model would be updated between iterations
        results = RasCheck.run_all(plan_number)
        
        progress.append({
            'Iteration': i + 1,
            'Total Messages': len(results.messages),
            'Errors': results.get_error_count(),
            'Warnings': results.get_warning_count()
        })
    
    return pd.DataFrame(progress)

# Simulate tracking (in practice, run after each model update)
progress_df = track_validation_progress("02", iterations=1)
print("Validation Progress Tracking:")
display(progress_df)

---

## Summary

This notebook demonstrated advanced RasCheck workflows:

### State Standards
- All 50 US states supported
- 7 states with stricter standards (IL: 0.1 ft, WI/MN/NJ/MI/IN/OH: 0.5 ft)
- Automatic limit lookup with `get_state_surcharge_limit()`

### Custom Thresholds
- Vegetated watershed configuration
- Urban development standards
- State-specific floodway limits
- Complete threshold customization

### Batch Processing
- Multiple threshold scenarios
- Automated CSV export
- Batch HTML report generation
- Comparative analysis

### Integration Patterns
- Pre-submission QA workflow
- Continuous monitoring during development
- Advanced CSV analysis

### Related Notebooks
- **300_quality_assurance_rascheck.ipynb** - Comprehensive RasCheck overview
- **301_advanced_structure_validation.ipynb** - Culvert and starting WSE checks